In [14]:
import requests
import pandas as pd
import numpy as np
from keys import *

city = "Tokyo"
country = "JPN"
response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')
print(response.json())

##En este codigo nos da como resultados la temperatura, la temperatura min, max, humedad, precion del aire, etc

{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1761782400, 'main': {'temp': 11.41, 'feels_like': 10.36, 'temp_min': 11.41, 'temp_max': 14.1, 'pressure': 1029, 'sea_level': 1029, 'grnd_level': 1028, 'humidity': 67, 'temp_kf': -2.69}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'clouds': {'all': 40}, 'wind': {'speed': 5.12, 'deg': 13, 'gust': 6.31}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': '2025-10-30 00:00:00'}, {'dt': 1761793200, 'main': {'temp': 13.15, 'feels_like': 12.04, 'temp_min': 13.15, 'temp_max': 16.64, 'pressure': 1029, 'sea_level': 1029, 'grnd_level': 1026, 'humidity': 58, 'temp_kf': -3.49}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'clouds': {'all': 27}, 'wind': {'speed': 3.63, 'deg': 23, 'gust': 3.86}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': '2025-10-30 03:00:00'}, {'dt': 1761804000, 'main': {'temp': 15.82, 'feels_like': 1

In [15]:
data = response.json()

# Extract forecast list
forecast_list = data.get('list', [])

# Prepare lists for DataFrame columns
times = []
temperatures = []
humidities = []
weather_statuses = []
wind_speeds = []
rain_volumes = []
snow_volumes = []

for entry in forecast_list:
    times.append(entry.get('dt_txt', np.nan))
    temperatures.append(entry.get('main', {}).get('temp', np.nan))
    humidities.append(entry.get('main', {}).get('humidity', np.nan))
    weather_statuses.append(entry.get('weather', [{}])[0].get('main', np.nan))
    wind_speeds.append(entry.get('wind', {}).get('speed', np.nan))
    rain_volumes.append(entry.get('rain', {}).get('3h', np.nan))
    snow_volumes.append(entry.get('snow', {}).get('3h', np.nan))

# Create DataFrame
df = pd.DataFrame({
    'weather_datetime': times,
    'temperature': temperatures,
    'humidity': humidities,
    'weather_status': weather_statuses,
    'wind': wind_speeds,
    'rain_qty': rain_volumes,
    'snow': snow_volumes,
    'municipality_iso_country': f"Tokyo,JPN"
})

print(df.head())

## Este codigo actua como el anterior pero esta vez nos los da como en forma de lista 


      weather_datetime  temperature  humidity weather_status  wind  rain_qty  \
0  2025-10-30 00:00:00        11.41        67         Clouds  5.12       NaN   
1  2025-10-30 03:00:00        13.15        58         Clouds  3.63       NaN   
2  2025-10-30 06:00:00        15.82        45         Clouds  3.13       NaN   
3  2025-10-30 09:00:00        17.69        42          Clear  3.89       NaN   
4  2025-10-30 12:00:00        15.99        55         Clouds  3.51       NaN   

   snow municipality_iso_country  
0   NaN                Tokyo,JPN  
1   NaN                Tokyo,JPN  
2   NaN                Tokyo,JPN  
3   NaN                Tokyo,JPN  
4   NaN                Tokyo,JPN  


In [16]:
import sqlalchemy
import pymysql
# connection details for the local mysql database
schema = "gans"
host = "127.0.0.1"
user = "root"
password = "Ua-xxx99"
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

ModuleNotFoundError: No module named 'sqlalchemy'